# Book 6 - Random Forest 10k data set

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import random
import datetime
import os

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patheffects as path_effects
from matplotlib import cm as cm2
matplotlib.style.use('ggplot') 
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn import linear_model, datasets
from sklearn.grid_search import GridSearchCV
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, \
    GradientBoostingClassifier, ExtraTreesClassifier, BaggingRegressor, \
    ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score \
#     confusion_matrix, roc_curve, roc_auc_score, precision_score, recall_score
from sklearn.cross_validation import train_test_split, KFold, cross_val_score


matplotlib.style.use('ggplot') 
%matplotlib inline

/Users/alex/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/alex/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# Bring in X & T train & test

In [2]:
# data = pd.read_csv('./data/data_bk1_complete.csv')
with open('./data/data_bk1_complete.pkl', 'r') as picklefile:
    data = pickle.load(picklefile)

In [3]:
# Use this to do train/test split after fit
display_list = list(data['display_id'].unique())

import random

# Random list from event_id list of df1 & df2
random.seed(a=44)
rand_list = random.sample(display_list,11000)

# these lists are for later splitting of train/test
train_list = rand_list[:10000]
test_list = rand_list[10000:]


In [4]:
# skinny down data to just what is needed before doing the fit
data = data[data['display_id'].isin(rand_list)]

In [5]:
data.tail(3)

,display_id,ad_id,clicked,uuid,document_id,platform,geo_location,country,state,DMA,date_local,period,day_of_mnth,hour,ads_per_page,ad_document_id,campaign_id,advertiser_id,doc_id_topic,ad_id_topic
77223,16874384,160764,0,a618ff8b112574,2673511,2,US>NJ>501,US,NJ,501,2016-06-26 23:59:45.144,overnight,26,23,4,1144499,18226,2553,275.0,260.0
77224,16874384,177111,0,a618ff8b112574,2673511,2,US>NJ>501,US,NJ,501,2016-06-26 23:59:45.144,overnight,26,23,4,1447390,21573,210,275.0,168.0
77225,16874384,262434,0,a618ff8b112574,2673511,2,US>NJ>501,US,NJ,501,2016-06-26 23:59:45.144,overnight,26,23,4,1621715,26260,2151,275.0,296.0


## Workaround for classifier data fit
If modeling on a larger scale, the data fit would include all states and all document pairs.

The scope of this woudl likely overload the local machine, so as a work around the data will be fit with only the combinations that are in the full data set.

For parameters that are not in the training set, they will have no impact on the classifier as they will all be zeros.

In [6]:
len(data)

56533

In [7]:
Y = data['clicked']
Y.head(2)

0    0
1    0
Name: clicked, dtype: int64

In [8]:
data.columns

Index([u'display_id', u'ad_id', u'clicked', u'uuid', u'document_id',
       u'platform', u'geo_location', u'country', u'state', u'DMA',
       u'date_local', u'period', u'day_of_mnth', u'hour', u'ads_per_page',
       u'ad_document_id', u'campaign_id', u'advertiser_id', u'doc_id_topic',
       u'ad_id_topic'],
      dtype='object')

In [9]:
disp_ID = data[['display_id','ad_id','clicked']]

In [10]:
disp_ID.head(2)

,display_id,ad_id,clicked
0,1525,8846,0
1,1525,45180,0


In [11]:
X = data[['display_id','platform', 'state', 'period', 'ads_per_page',
            'campaign_id', 'advertiser_id', 'doc_id_topic','ad_id_topic']]

In [12]:
# change numbers to strings before get_dummies

to_num_str = ['platform', 'ads_per_page', 'campaign_id', 
              'advertiser_id', 'doc_id_topic','ad_id_topic']

for s in to_num_str:
    print(s)
    X[s] = X[s].apply(str)

platform
ads_per_page
campaign_id
advertiser_id
doc_id_topic
ad_id_topic


/Users/alex/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
start = datetime.datetime.now()

X = pd.get_dummies(X, prefix=['platform', 'state', 'period', 'ads_per_page',
            'campaign_id', 'advertiser_id', 'doc_id_topic','ad_id_topic'])


finish = datetime.datetime.now()
print(finish - start)

0:00:13.380235


In [14]:
X.head(1)

,display_id,platform_1,platform_2,platform_3,state_AK,state_AL,state_AR,state_AZ,state_CA,state_CO,...,ad_id_topic_91.0,ad_id_topic_92.0,ad_id_topic_93.0,ad_id_topic_94.0,ad_id_topic_95.0,ad_id_topic_96.0,ad_id_topic_97.0,ad_id_topic_98.0,ad_id_topic_99.0,ad_id_topic_nan
0,1525,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
X = X.drop(['platform_3','state_AK','period_morning','campaign_id_18226','advertiser_id_2553',
       'doc_id_topic_275.0','ad_id_topic_91.0','ads_per_page_12'],1)

In [16]:
X.head(2)

,display_id,platform_1,platform_2,state_AL,state_AR,state_AZ,state_CA,state_CO,state_CT,state_DC,...,ad_id_topic_90.0,ad_id_topic_92.0,ad_id_topic_93.0,ad_id_topic_94.0,ad_id_topic_95.0,ad_id_topic_96.0,ad_id_topic_97.0,ad_id_topic_98.0,ad_id_topic_99.0,ad_id_topic_nan
0,1525,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1525,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
X_train = X[X['display_id'].isin(train_list)]
X_test = X[X['display_id'].isin(test_list)]

In [18]:
len(X_train)

51340

In [19]:
Y_train = list(disp_ID[disp_ID['display_id'].isin(train_list)]['clicked'])

In [20]:
Y_test = list(disp_ID[disp_ID['display_id'].isin(test_list)]['clicked'])

In [21]:
disp_ID_train = disp_ID[disp_ID['display_id'].isin(train_list)]
disp_ID_test = disp_ID[disp_ID['display_id'].isin(test_list)]

In [22]:
print('X_train length: {0:,}'.format(len(X_train)))
print('X_test  length:  {0:,}'.format(len(X_test)))

X_train length: 51,340
X_test  length:  5,193


In [23]:
print('Y_train length: {0:,}'.format(len(Y_train)))
print('Y_test  length:  {0:,}'.format(len(Y_test)))

Y_train length: 51,340
Y_test  length:  5,193


In [24]:
print('disp_ID_train length: {0:,}'.format(len(disp_ID_train)))
print('disp_ID_test  length:  {0:,}'.format(len(disp_ID_test)))

disp_ID_train length: 51,340
disp_ID_test  length:  5,193


# Logistic Regresion

Log Reg default settings (C=1)
* correct predictions: 323
* Precent correct: 32.3%

Log Reg max precision settings (C=5)
* correct predictions: 
* Precent correct: 

Log Reg accuracy settings (C=10)
* correct predictions:
* Precent correct: 


In [25]:
lg = LogisticRegression()

lg.fit(X_train,Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [26]:
# get probabilities of clicked/not clicked
lg_proba = lg.predict_proba(X_test)
lg_Y_proba = pd.DataFrame(lg_proba,columns = ['p_no_click','p_click'])

In [27]:
Disp_test = list(disp_ID_test['display_id'])
len(Disp_test)

5193

In [28]:
ppp = lg_Y_proba
# post processing of predictor

In [29]:
len(ppp)

5193

In [30]:
ppp['display_id'] = Disp_test

In [31]:
ppp.head()

,p_no_click,p_click,display_id
0,0.500655,0.499345,19234
1,0.500655,0.499345,19234
2,0.500655,0.499345,19234
3,0.500655,0.499345,19234
4,0.500655,0.499345,19234


In [32]:
disp_ID_test.head()

,display_id,ad_id,clicked
71,19234,89726,0
72,19234,173384,0
73,19234,173400,1
74,19234,180960,0
75,19234,225440,0


## Reduce to 1-click per display id

In [33]:
checker = ppp
checker.head(3)

,p_no_click,p_click,display_id
0,0.500655,0.499345,19234
1,0.500655,0.499345,19234
2,0.500655,0.499345,19234


In [34]:
checker['Y_test'] = pd.Series(Y_test)
checker['prediction'] = [0 for _ in range(len(checker))] # puts zeros in place
checker.head()

,p_no_click,p_click,display_id,Y_test,prediction
0,0.500655,0.499345,19234,0,0
1,0.500655,0.499345,19234,0,0
2,0.500655,0.499345,19234,1,0
3,0.500655,0.499345,19234,0,0
4,0.500655,0.499345,19234,0,0


In [35]:
pages = checker['display_id'].unique()
len(pages)

1000

In [36]:
# system to use probabilities to choose which page is clicked
# compensates for zero probability across the board, or muliple matching high probabilities

for d_num in pages:  
    indx = 0 # index that becomes click
    disp_list = checker[checker['display_id'] == d_num].index.tolist()

    temp = pd.DataFrame(ppp[ppp['display_id'] == d_num])

    p_max = temp['p_click'].max()

    if p_max == 0:
        indx = random.choice(disp_list) # randomly picks from all matched values
    else:
        counter = 0
        short_list =[]
        for index, row in temp.iterrows():
            if row['p_click'] == p_max:
                counter += 1
                short_list.append(index)
        indx = random.choice(short_list) # randomly picks from all matched values
    checker.set_value(indx, 'prediction', 1)


In [37]:
sum(checker['Y_test'])

1000

In [38]:
sum(checker['prediction'])

1000

In [39]:
count = 0
for index, row in checker.iterrows():
    if row['prediction'] == 1 and row['Y_test'] == 1:
        count +=1
print('Clicks correctly predicted: {0:,}'.format(count))
print('Percent correctly predicted: {0:%}'.format(float(count)/sum(checker['Y_test'])))

Clicks correctly predicted: 323
Percent correctly predicted: 32.300000%


In [40]:
grid = GridSearchCV(lg, scoring = "precision",
                    param_grid = {'penalty':['l1','l2']})
grid.fit(X_train,Y_train)


#look at max features

print grid.best_estimator_
print grid.best_score_
os.system('say "I am finished!"')

/Users/alex/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
0.533209082293


0

In [41]:
grid.best_estimator_

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)